In [27]:
from google.colab import drive
drive.mount ('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
import warnings
warnings.filterwarnings('ignore')

In [29]:
%cd '/content/drive/My Drive/Colab Notebooks/2-parametro-ventas-insumos'

/content/drive/My Drive/Colab Notebooks/2-parametro-ventas-insumos


In [30]:
import os
import random as rd
import numpy as np
import pandas as pd

In [31]:
ruta_iniciales = os.path.join(os.getcwd(), 'archivos-iniciales')
ruta_resultados = os.path.join(os.getcwd(), 'archivos-resultados')

In [32]:
archivos_iniciales = os.listdir(ruta_iniciales)
archivos_iniciales

['reps-ids-cacao.csv',
 'reps-ids-maiz.csv',
 'reps-ids-arroz.csv',
 'arroz.csv',
 'maiz.csv',
 'cacao.csv']

IDS UNICOS

In [33]:
arroz = pd.read_csv(os.path.join(ruta_iniciales, 'arroz.csv'))

IDS REPETIDOS

In [34]:
arroz_reps = pd.read_csv(os.path.join(ruta_iniciales, 'reps-ids-arroz.csv'))

<hr>

transitorios:
  > [arroz, maiz]

permanentes
  > [cacao]

'Ventas' para arroz y maiz

'ventas' para cacao

### CORRIGIENDO LAS UNIDADES

FUNCIONES

In [35]:
def conversion_tonelada_litro(df, columna_unidades, columna_valor):
  mascara = df[columna_unidades] == 4
  df[columna_valor][mascara] = df[columna_valor][mascara] * 2831.6846592
  return df  

def conversion_quintal_litro(df, columna_unidades, columna_valor):
  mascara = df[columna_unidades] == 3
  df[columna_valor][mascara] = df[columna_valor][mascara] * 100 * 0.45359236996888618876
  return df

def conversion_kilogramo_litro(df, columna_unidades, columna_valor):
  mascara = df[columna_unidades] == 2
  df[columna_valor][mascara] = df[columna_valor][mascara] * 1
  return df

def conversion_libra_litro(df, columna_unidades, columna_valor):
  mascara = df[columna_unidades] == 1
  df[columna_valor][mascara] = df[columna_valor][mascara] * 0.45359236996888618876
  return df

In [36]:
arroz_reps = conversion_tonelada_litro(arroz_reps, 'ct_umed_npk_fq', 'ct_cantidad_npk_fq')
arroz_reps = conversion_quintal_litro(arroz_reps, 'ct_umed_npk_fq', 'ct_cantidad_npk_fq')
arroz_reps = conversion_kilogramo_litro(arroz_reps, 'ct_umed_npk_fq', 'ct_cantidad_npk_fq')
arroz_reps = conversion_libra_litro(arroz_reps, 'ct_umed_npk_fq', 'ct_cantidad_npk_fq')

arroz = conversion_tonelada_litro(arroz, 'ct_umed_npk_fq', 'ct_cantidad_npk_fq')
arroz = conversion_quintal_litro(arroz, 'ct_umed_npk_fq', 'ct_cantidad_npk_fq')
arroz = conversion_kilogramo_litro(arroz, 'ct_umed_npk_fq', 'ct_cantidad_npk_fq')
arroz = conversion_libra_litro(arroz, 'ct_umed_npk_fq', 'ct_cantidad_npk_fq')

### FUNCIONES

In [37]:
def parametro_ventas(df_repetidos, df_unicos, col_superficie, col_ventas):
  ids = df_unicos['Identificador']
  df_unicos.set_index('Identificador', inplace=True)
  df_unicos['parametros_ventas'] = 0
  for id in ids:
    mascara = df_repetidos['Identificador'] == id
    
    array_superficie_cos = df_repetidos[mascara][col_superficie].values
    array_superficie_cos = np.nan_to_num(array_superficie_cos, nan=0)
    superficie_cosechada = array_superficie_cos.sum()
    
    array_ventas = df_repetidos[mascara][col_ventas].values
    array_ventas = np.nan_to_num(array_ventas, nan=0)
    ventas = array_ventas.sum()

    division = ventas / superficie_cosechada
    df_unicos.loc[id, 'parametro_ventas'] = division
  
  df_unicos.reset_index(inplace=True)
  return df_unicos


def parametro_insumos(df_repetidos, df_unicos, col_superficie, col_insumos):
  ids = df_unicos['Identificador']
  df_unicos.set_index('Identificador', inplace=True)
  df_unicos['parametros_ventas'] = 0
  for id in ids:
    mascara = df_repetidos['Identificador'] == id
    
    array_superficie_plt = df_repetidos[mascara][col_superficie].values
    array_superficie_plt = np.nan_to_num(array_superficie_plt, nan=0)
    superficie_plantada = array_superficie_plt.sum()
    
    array_insumos = df_repetidos[mascara][col_insumos].values
    array_insumos = np.nan_to_num(array_insumos, nan=0)
    insumos = array_insumos.sum()

    division = insumos / superficie_plantada
    df_unicos.loc[id, 'parametro_insumos'] = division
  
  df_unicos.reset_index(inplace=True)
  return df_unicos

### arroz

In [38]:
# muestra = rd.sample(list(arroz_reps['Identificador'][arroz_reps['Identificador'].duplicated()]), 5)
muestra = [9185002042830003, 9205002087850001, 9025002076870001, 9065202068980004, 9025002072270008]
muestra

[9185002042830003,
 9205002087850001,
 9025002076870001,
 9065202068980004,
 9025002072270008]

In [39]:
arroz_reps[arroz_reps['Identificador'].isin(muestra)][['Identificador', 'ct_k511', 'ct_k509', 'Ventas', 'ct_cantidad_npk_fq']]

,Identificador,ct_k511,ct_k509,Ventas,ct_cantidad_npk_fq
215,9205002087850001,100.00,11.0,500.000000,15875.732949
216,9205002087850001,100.00,6.0,500.000000,15875.732949
700,9025002076870001,45.00,8.0,200.000000,NaN
701,9025002076870001,44.25,12.0,200.000000,NaN
880,9025002072270008,6.00,10.0,30.000000,NaN
881,9025002072270008,12.99,10.0,60.000000,NaN
990,9065202068980004,2.45,11.0,14.181818,300.000000
991,9065202068980004,2.45,8.0,15.600000,300.000000
992,9065202068980004,2.00,11.0,7.090909,200.000000
993,9065202068980004,2.00,5.0,8.272727,200.000000


In [40]:
arroz = parametro_ventas(arroz_reps, arroz, 'ct_k511', 'Ventas')
arroz = parametro_insumos(arroz_reps, arroz, 'ct_k509', 'ct_cantidad_npk_fq')

In [41]:
arroz[arroz['Identificador'].isin(muestra)][['Identificador', 'ct_k511', 'ct_k509', 'Ventas', 'parametro_insumos', 'parametro_ventas']]

,Identificador,ct_k511,ct_k509,Ventas,parametro_insumos,parametro_ventas
135,9205002087850001,100.00,11.0,1000.000000,1867.733288,5.000000
449,9025002076870001,45.00,8.0,400.000000,0.000000,4.481793
556,9025002072270008,6.00,10.0,90.000000,0.000000,4.739336
624,9065202068980004,2.45,11.0,45.145455,28.571429,5.072523
747,9185002042830003,1.00,12.0,8.590909,15.119746,4.295455


In [42]:
arroz['parametro_insumos'].replace([np.inf, -np.inf], np.nan, inplace=True)
arroz['parametro_ventas'].replace([np.inf, -np.inf], np.nan, inplace=True)
arroz[arroz['Identificador'].isin(muestra)][['Identificador', 'ct_k511', 'ct_k509', 'Ventas', 'parametro_insumos', 'parametro_ventas']]

,Identificador,ct_k511,ct_k509,Ventas,parametro_insumos,parametro_ventas
135,9205002087850001,100.00,11.0,1000.000000,1867.733288,5.000000
449,9025002076870001,45.00,8.0,400.000000,0.000000,4.481793
556,9025002072270008,6.00,10.0,90.000000,0.000000,4.739336
624,9065202068980004,2.45,11.0,45.145455,28.571429,5.072523
747,9185002042830003,1.00,12.0,8.590909,15.119746,4.295455


### GUARDADO

In [43]:
arroz.to_csv(os.path.join(ruta_resultados, 'arroz.csv'), index=False, encoding='utf-8')

In [44]:
arroz.to_stata(os.path.join(ruta_resultados, 'arroz.dta'), write_index=False)